In [1]:
from sqlalchemy import create_engine
from pathlib import Path
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format


db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
with engine.connect() as conn, conn.begin():  
    cobranza_enero = pd.read_sql_table('cobranza_enero', conn, dtype_backend='numpy_nullable')
    cobranza_febrero = pd.read_sql_table('cobranza_febrero', conn, dtype_backend='numpy_nullable')
    cobranza_marzo = pd.read_sql_table('cobranza_marzo', conn, dtype_backend='numpy_nullable')
    cobranza_abril = pd.read_sql_table('cobranza_abril', conn, dtype_backend='numpy_nullable')
    cobranza_mayo = pd.read_sql_table('cobranza_mayo', conn, dtype_backend='numpy_nullable')
    cobranza_año = pd.read_sql_table('cobranza_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [6]:
cobranza_año[['pay_journal','pay_amount']].groupby('pay_journal').sum()

,pay_amount
pay_journal,
Banco Bana70103464895,"127,059.52"
Banco Sant65507231316,"14,024,739.13"
Banco Sant92000702524,"1,844,550.51"
Banco Scot23900002860,"59,750.38"
Efectivo,"2,614.98"
Efectivo A1,"225,386.36"
Efectivo A2,"5,085.44"
Miscellaneous Operations,836.92
Punto De Venta A1,"5,128,422.64"


In [8]:
cobranza_año[cobranza_año['pay_journal'] == 'Miscellaneous Operations']

,fact_doc_id,fact_doc_name,partner_id,partner_name,fact_doc_date,fact_doc_total,fact_doc_deuda,pay_journal,ref,pay_amount,pay_date,dias_pago,pay_pos,salesperson_id,salesperson_name,sale_team_id,sale_team_description,business_model,warehouse
2621,14272,F2-CC/2024/01862,13732,FRANCISCO LOPEZ SEGOVIA,2024-01-27,985.30,0.00,Miscellaneous Operations,<NA>,836.35,2024-01-31,4,<NA>,227,Patricia Flores Pantaleón,6,A2 Piso,Piso,A2
8140,42702,F1-CC/2024/03839,16253,Mostrador,2024-03-14,217.96,0.00,Miscellaneous Operations,<NA>,0.57,2024-03-31,17,<NA>,218,Gladiz Melisa Galvez Espinoza,5,A1 Piso,Piso,A1


In [12]:
cobranza_año[cobranza_año['pay_journal'] == 'Efectivo']

,fact_doc_id,fact_doc_name,partner_id,partner_name,fact_doc_date,fact_doc_total,fact_doc_deuda,pay_journal,ref,pay_amount,pay_date,dias_pago,pay_pos,salesperson_id,salesperson_name,sale_team_id,sale_team_description,business_model,warehouse
4481,26717,F2-VS/2024/00372,13544,PROMOTORA TURISTICA NAUTILIUS,2024-02-17,139.10,0.00,Efectivo,<NA>,139.10,2024-02-17,0,<NA>,205,Brenda Luz Acosta Lopez,8,A2 CE,CE,A2
4489,26732,F2-VS/2024/00376,13170,ASOCIACION DE CONDOMINOS CORAL BAJA,2024-02-17,139.72,0.00,Efectivo,<NA>,139.72,2024-02-17,0,<NA>,205,Brenda Luz Acosta Lopez,8,A2 CE,CE,A2
6177,30201,F1-VS/2024/00636,16588,AMX CABO HOLDINGS,2024-02-23,699.35,0.00,Efectivo,<NA>,699.35,2024-02-26,3,<NA>,210,Irma Carvajal Flores,7,A1 CE,CE,A1
7080,23970,F2-VS/2024/00331,13156,ALMAX CABO,2024-02-12,241.36,0.00,Efectivo,<NA>,241.36,2024-02-20,8,<NA>,205,Brenda Luz Acosta Lopez,8,A2 CE,CE,A2
7265,24398,F1-VS/2024/00523,15227,COMERCIALIZADORA VISTA BALLENA,2024-02-13,321.38,0.00,Efectivo,<NA>,321.38,2024-02-13,0,<NA>,211,Mayra Angelica Parada Manjarrez,7,A1 CE,CE,A1
10556,39016,F1-VS/2024/00818,15227,COMERCIALIZADORA VISTA BALLENA,2024-03-08,"1,074.07",0.00,Efectivo,<NA>,"1,074.07",2024-03-09,1,<NA>,211,Mayra Angelica Parada Manjarrez,7,A1 CE,CE,A1
